In [ ]:
import pandas as pd

## Preprocessing Data
# Reading Data


data1 = '/content/drive/MyDrive/Colab Notebooks/MLmodel_djlee/LEV3/043_200421_Santafe.csv'
data2 = '/content/drive/MyDrive/Colab Notebooks/MLmodel_djlee/LEV3/045_200422_Santafe.csv'
data3 = '/content/drive/MyDrive/Colab Notebooks/MLmodel_djlee/LEV3/046_200423_Santafe.csv'
data4 = '/content/drive/MyDrive/Colab Notebooks/MLmodel_djlee/LEV3/047_200424_Santafe.csv'

dataFrame_raw = pd.concat(map(pd.read_csv, [data1, data2, data3, data4]), ignore_index=True)

output_variable_names = [
    'CAL_CO2Flowrate_gphr'
    #'CAL_NOxFlowrateLNTOutletValid_gps'
]

variable_names_to_extract = [ 
#   'AUX_NOxLNInlet_ppm', 
#   'AUX_NOxLNTOutlet_ppm',
#   'CAL_Time_sec', 
#   'AUX_RTCDate_YYYYMMDD', 
    'OBD_CalEngLoad_perc',
#   'OBD_EngCoolantTemp_degC', 
    'OBD_EngineSpeed_rpm', 
#   'OBD_VehicleSpeed_kmph', 
#   'OBD_ActEngPerTorque_perc',
#   'OBD_EngRefTorque_Nm', 
#   'OBD_MAFSensor_gps', 
#   'OBD_EGT11_degC', 
#   'OBD_EGT12_degC',
#   'OBD_DPFDiffPress_kPa', 
#   'AUX_NOxSCROutlet_ppm', 
#   'AUX_lambdaLNTInlet_none', 
#   'AUX_lambdaLNTOutlet_none',
#   'AUX_lambdaSCROutlet_none', 
#   'AUX_O2FracLNTInlet_volPerc', 
#   'AUX_O2FracLNTOutlet_volPerc',
#   'AUX_O2FracSCROutlet_volPerc',
#   'AUX_SensAmbTemp_degC',
#   'AUX_SensAmbRH_perc',
#   'AUX_SensBaroPress_kPa',
#   'AUX_SensTempTurbinOutlet_degC',
#   'AUX_SensTempInManiInlet_degC',
#   'AUX_SensTempTurbinInlet_degC',
#   'AUX_SensTempLNTOutlet_degC',
#   'AUX_SensTempSCROutlet_degC',
#   'AUX_SensTempCompressorOutlet_degC',
#   'AUX_SensPressEGRCoolerInlet_absBar',
#   'AUX_SensDiffPressDPF_kPa',
#   'AUX_SensDiffPressSCR_kPa',
#   'CAL_ExhFlowrate_gps',
    'CAL_CO2Flowrate_gphr',
#   'CAL_NOxFlowrateLNTInletValid_gps',
#   'CAL_NOxFlowrateLNTOutletValid_gps',
#   'CAL_NOxFlowrateSCROutletValid_gps'
]

dataFrame = dataFrame_raw[[
    'OBD_CalEngLoad_perc',
    #'OBD_EngCoolantTemp_degC', 
    'OBD_EngineSpeed_rpm', 
    'CAL_CO2Flowrate_gphr'
]]

#print(dataFrame.head())

# Cleaning data
RPM_min = 100                      # excluding engine-stop
NOx_max = 1649                     # excluding clipped NOx data (since the sensor maxed out at 1650 ppm)

dataFrame = dataFrame.loc[(dataFrame['OBD_EngineSpeed_rpm'] >= RPM_min)] #& (dataFrame['AUX_NOxLNInlet_ppm'] <= NOx_max))]
#dataFrame = dataFrame.drop(['AUX_NOxLNInlet_ppm'], axis=1)
dataFrame = dataFrame.dropna() 

x = dataFrame.drop(labels='CAL_CO2Flowrate_gphr',axis=1)
y = dataFrame[['CAL_CO2Flowrate_gphr']]
#print(x.info())

x_data = x.to_numpy()
y_data = y.to_numpy()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(x_data, y_data, test_size=0.25, random_state=42)

from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias=False) 
poly.fit(train_input)
train_poly = poly.transform(train_input)   
test_poly = poly.transform(test_input)

## 정규화
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_poly)

train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

#print(train_scaled.shape)

## 릿지 회기
from sklearn.linear_model import Ridge

train_score = []
test_score = []

alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
for alpha in alpha_list:
    # 릿지 모델을 만듭니다
    ridge = Ridge(alpha=alpha)
    # 릿지 모델을 훈련합니다
    ridge.fit(train_scaled, train_target)
    # 훈련 점수와 테스트 점수를 저장합니다
    train_score.append(ridge.score(train_scaled, train_target))
    test_score.append(ridge.score(test_scaled, test_target))

# 릿지 모델의 최적의 alpha값을 사용하여 모델 훈련 후 결과 도출
ridge = Ridge(alpha=0.1)
ridge.fit(train_scaled, train_target)

print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))

In [ ]:
## 라쏘 회기
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(train_scaled, train_target)

train_score = []
test_score = []

alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
for alpha in alpha_list:
    # 라쏘 모델을 만듭니다
    lasso = Lasso(alpha=alpha, max_iter=10000)
    # 라쏘 모델을 훈련합니다
    lasso.fit(train_scaled, train_target)
    # 훈련 점수와 테스트 점수를 저장합니다
    train_score.append(lasso.score(train_scaled, train_target))
    test_score.append(lasso.score(test_scaled, test_target))

# 라쏘 모델의 최적의 alpha값을 사용하여 모델 훈련 후 결과 도출
lasso = Lasso(alpha=10)
lasso.fit(train_scaled, train_target)

print(lasso.score(train_scaled, train_target))
print(lasso.score(test_scaled, test_target))
